In [1]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

import os
import sys

In [4]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

In [5]:
!mkdir pdfs

In [6]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [7]:
print(data)

[Document(page_content=' \n \n \n \n \n \n \nData  Science  \nInterview  Questions  \n(30 days of Interview  Preparation)', metadata={'source': 'pdfs/Day1.pdf', 'page': 0}), Document(page_content='INEURON.AI  \n \n Page 2 \n  \nQ1.  What is the difference between AI,  Data  Science , ML, and DL ? \n \nAns 1 :  \n         \n  \n \nArtificial Intelligence : AI is purely math and scientific exercis e, but when it became computa tional , it \nstarted to solve human problems formalized into a subset of computer science. A rtificial intelligence has \nchanged the original computational statistics paradigm to the modern idea that machines could mimic \nactual human capabilities, such as deci sion making and perfo rming more “human” tasks. Modern AI into \ntwo categories  \n1. General AI - Planning, decision making, identifying objects, recognizing sounds, social & \nbusiness transactions  \n2. Applied AI - driverless/ Autonomous car or machine smartly trade st ocks \n \nMachine Learning : Ins

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20
)

In [9]:
docs = text_splitter.split_documents(data)

In [10]:
print(len(docs))

103


In [11]:
print(docs[0])

page_content='Data  Science  \nInterview  Questions  \n(30 days of Interview  Preparation)' metadata={'source': 'pdfs/Day1.pdf', 'page': 0}


In [12]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [13]:
query = embeddings.embed_query("Hello World")

In [14]:
print("Length",len(query))

Length 384


In [15]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','814b0486-9f63-4cc8-aea6-abaefa4b0d7b')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','gcp-starter')

In [17]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name = "anantha"

In [19]:
docsearch = Pinecone.from_texts([t.page_content for t in docs],embeddings,index_name=index_name)

In [21]:
#docsearch = Pinecone.from_existing_index(index_name, embeddings)
#if you have already have an index

In [22]:
query = "What is Data Science?"


In [23]:
docs = docsearch.similarity_search(query,k=4)

In [25]:
docs

[Document(page_content='Data Science : Data science ha s many tools, techniques, and algorithms called from these fields, plus \nothers –to handle big data  \nThe goal of data science, somewhat similar to machine learning, is to make accurate predictions and to \nautomate and perform transactions in real -time, such as purchasing internet traffic or automatically \ngenerating content.'),
 Document(page_content='INEURON.AI  \n \n Page 3 \n Data science relies less on math and coding and more on data and building new systems to process the \ndata. Relying on the fields of data integration, distributed architecture, aut omated machine learning, data \nvisualization, data engineering, and automated data -driven decisions, data science can cover an entire \nspectrum of data processing, not only the algorithms or statistics related to data.  \n \nDeep Learning : It is a technique for i mplementing ML.'),
 Document(page_content='data. Cleaning the data , handling the missing data if any , dat

In [26]:
from huggingface_hub import notebook_login
import torch

In [27]:
notebook_login()

In [28]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [29]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    torch_dtype=torch.float16,
    use_auth_token=True,
    load_in_8bit=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [30]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=512,
    do_sample=True,
    top_k=30,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [31]:
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={'temperature':0.1}
)

In [32]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

In [33]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [34]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [39]:
SYSTEM_PROMPT

"<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n"

In [36]:
instruction = """
{context}

Question: {question}
"""

In [37]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [38]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [40]:
prompt = PromptTemplate(
    template=template,
    input_variables=['context','question']
)

In [42]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':prompt}
)

In [43]:
result = qa_chain("What is Data Science?")

In [44]:
result['result']

"  Based on the provided context, here's my answer to your question:\nData Science is a field that encompasses a wide range of techniques, tools, and algorithms used to handle and process large amounts of data. The goal of data science is to make accurate predictions and automate transactions in real-time, such as purchasing internet traffic or generating content. While data science relies on data and building new systems to process the data, it also draws on the fields of data integration, distributed architecture, automatic machine learning, data visualization, data engineering, and automated data-driven decisions. In other words, data science goes beyond traditional machine learning and statistics by covering an entire spectrum of data processing.\nI hope this helps! Let me know if you have any further questions."

In [45]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")

prompt:what is Machine Learning?
Answer:  Based on the provided context, the answer to the question "what is Machine Learning?" is:
I don't know.

Machine Learning is a field of study that involves developing algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying on patterns and inference instead. However, I don't have enough information to provide a clear definition of Machine Learning. The context provided mentions types of Machine Learning, such as supervised and unsupervised learning, but it does not provide a clear explanation of what Machine Learning is.
prompt:exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
